In [5]:
import fastf1 as ff1
import pandas as pd
ff1.Cache.enable_cache('cache')


In [55]:
session = ff1.get_session(2022, 'Monaco', 'Q')
session.load()

In [74]:
session.laps.pick_driver('BOT').telemetry

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
2,2022-05-28 14:00:20.973,0 days 00:15:20.579000,,89.581944,0 days 00:00:00,4662,39,1,3,False,8,interpolation,0.096674,0.000002,OnTrack,-7554,-6709,0
3,2022-05-28 14:00:21.014,0 days 00:15:20.620000,,89.581944,0 days 00:00:00.041000,4740,40,1,3,False,8,pos,0.574829,0.000010,OnTrack,-7555,-6698,0
4,2022-05-28 14:00:21.017,0 days 00:15:20.623000,,89.581944,0 days 00:00:00.044000,4896,41,1,4,False,8,car,0.610000,0.000011,OnTrack,-7555,-6697,0
5,2022-05-28 14:00:21.174,0 days 00:15:20.780000,,89.581944,0 days 00:00:00.201000,5005,42,1,4,False,8,pos,2.486341,0.000045,OnTrack,-7568,-6668,0
6,2022-05-28 14:00:21.377,0 days 00:15:20.983000,4,89.581944,0 days 00:00:00.404000,5114,44,1,5,False,8,car,5.010000,0.000091,OnTrack,-7575,-6649,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21141,2022-05-28 14:46:59.424,0 days 01:01:59.030000,11,2934.388056,0 days 00:46:38.451000,5873,48,1,7,False,8,car,55305.170833,0.999808,OnTrack,-6401,-9743,511
21142,2022-05-28 14:46:59.681,0 days 01:01:59.287000,11,2936.554722,0 days 00:46:38.708000,6094,48,1,10,False,8,pos,55308.633429,0.999870,OnTrack,-6372,-9708,512
21143,2022-05-28 14:46:59.824,0 days 01:01:59.430000,11,2938.721389,0 days 00:46:38.851000,6316,49,1,14,False,8,car,55310.615278,0.999906,OnTrack,-6388,-9701,512
21144,2022-05-28 14:46:59.861,0 days 01:01:59.467000,11,2938.721389,0 days 00:46:38.888000,6288,50,1,21,False,8,pos,55311.135813,0.999915,OnTrack,-6394,-9700,512


In [75]:
session.weather_data

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:59.917000,27.6,37.0,1004.0,False,55.3,344,0.8
1,0 days 00:01:59.913000,27.6,37.0,1004.0,False,55.3,3,0.8
2,0 days 00:02:59.934000,27.6,38.0,1004.0,False,54.8,328,0.6
3,0 days 00:03:59.929000,27.6,39.0,1004.0,False,54.8,313,1.0
4,0 days 00:04:59.925000,27.5,39.0,1004.2,False,53.0,303,1.1
...,...,...,...,...,...,...,...,...
76,0 days 01:17:00.275000,26.5,42.0,1003.2,False,45.6,0,0.8
77,0 days 01:18:00.300000,26.6,42.0,1003.2,False,45.3,0,0.3
78,0 days 01:19:00.304000,26.7,44.0,1003.0,False,44.7,176,0.5
79,0 days 01:20:00.309000,26.6,43.0,1003.2,False,44.5,0,1.0
